# FinanceBot Workflow
The purpose of this file is to ensure that all dependencies are correctly installed.

In [ ]:
 # from _setup import *

In [1]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain_community.tools import TavilySearchResults
import yfinance as yf
import matplotlib.pyplot as plt
import io
import base64

def fetch_stock_data(ticker: str, period: str = "14d"):
    """Fetches historical stock data for the given ticker over the specified period."""
    stock = yf.Ticker(ticker)
    data = stock.history(period=period)
    return data

def plot_stock_data(data, ticker: str):
    """Generates a base64-encoded plot of the stock data."""
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data['Close'], label='Closing Price', marker='o')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.title(f'{ticker} Stock Price Over the Last 2 Weeks')
    plt.legend()
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    encoded_string = base64.b64encode(buf.getvalue()).decode('utf-8')
    buf.close()
    
    return encoded_string

def get_market_insights(ticker: str):
    """Uses TavilySearchResults to gather recent market insights related to the stock."""
    search = TavilySearchResults()
    results = search.run(f"Latest news and analysis on {ticker} stock performance")
    return results

# Define tools for LangChain agent
stock_data_tool = Tool(
    name="Stock Data Fetcher",
    func=fetch_stock_data,
    description="Fetches the last 2 weeks of stock data for a given ticker."
)

stock_plot_tool = Tool(
    name="Stock Data Plotter",
    func=plot_stock_data,
    description="Plots the stock data and returns a base64 image string."
)

market_insights_tool = Tool(
    name="Market Insights Retriever",
    func=get_market_insights,
    description="Fetches recent news and analysis for a given stock ticker."
)

# Initialize the LangChain agent
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    tools=[stock_data_tool, stock_plot_tool, market_insights_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\Trevo\AppData\Local\Temp\ipykernel_17480\3118412726.py:59: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)
C:\Users\Trevo\AppData\Local\Temp\ipykernel_17480\3118412726.py:60: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.

In [2]:
fetch_stock_data("FORD")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-22 00:00:00-05:00,5.99,6.80,5.99,6.47,28900,0.0,0.0
2025-01-23 00:00:00-05:00,6.60,6.96,6.60,6.94,24100,0.0,0.0
2025-01-24 00:00:00-05:00,6.91,6.99,6.41,6.54,18900,0.0,0.0
2025-01-27 00:00:00-05:00,6.43,6.59,6.07,6.17,9000,0.0,0.0
2025-01-28 00:00:00-05:00,6.24,6.59,5.94,5.96,43800,0.0,0.0
2025-01-29 00:00:00-05:00,6.00,6.19,6.00,6.11,5000,0.0,0.0
2025-01-30 00:00:00-05:00,6.11,6.20,5.56,5.66,32200,0.0,0.0
2025-01-31 00:00:00-05:00,6.00,6.25,5.83,6.01,23600,0.0,0.0
2025-02-03 00:00:00-05:00,6.68,6.70,6.03,6.46,24100,0.0,0.0


In [7]:
# Define the query
query = "What is the latest stock data and market insights for FORD?"

# Run the agent with the query
response = agent.run(query)

# Print the response
print(response)



> Entering new AgentExecutor chain...
I should first fetch the latest stock data for FORD and then retrieve market insights for the same.
Action: Stock Data Fetcher
Action Input: ticker='FORD', period='14d'

$TICKER='FORD', PERIOD='14D': possibly delisted; no price data found  (period=14d) (Yahoo error = "No data found, symbol may be delisted")



Observation: Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Thought:It seems that there is no recent stock data available for FORD. I should try fetching market insights instead.
Action: Market Insights Retriever
Action Input: ticker='FORD'
Observation: [{'url': 'https://stockanalysis.com/stocks/f/', 'content': 'Ford Motor Company (F) Stock Price, Quote & News - Stock Analysis Ford Motor Company (F) Kevin Roberts (@cargurus) notes uncertainty around Trump tariffs as a "cloud hanging over the auto industry." That was seen clearly on Ford Motor Company\'s (F) earnings call Wednesday where they expre... Ford Motor Co (NYSE: F) chief executive Jim Farley is concerned that President Trump\'s planned tariffs could do more harm than good to the US automotive industry. Ford Motor Company (F) Q4 2024 Earnings Call Transcript Ford Motor Company (NYSE:F) Q4 2024 Earnings Conference Call February 5, 2025 5:00 PM ET Company Participants Lynn Antipas Tyson - Executiv

In [5]:
# Print the response
pprint.pprint(response)

('The latest market insights for FORD indicate concerns about Trump tariffs '
 'and underwhelming fourth-quarter results.')
